In [1]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

In [2]:
import nltk
import re
import string
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer 
from nltk.stem import WordNetLemmatizer 

In [3]:
data=pd.read_csv(r'C:\Users\Suneetha\python files\PROJECT-2\hotel_reviews.csv',encoding="Latin1")
data

,Review,Rating
0,nice hotel expensive parking got good deal sta...,4
1,ok nothing special charge diamond member hilto...,2
2,nice rooms not 4* experience hotel monaco seat...,3
3,"unique, great stay, wonderful time hotel monac...",5
4,"great stay great stay, went seahawk game aweso...",5
...,...,...
20486,"best kept secret 3rd time staying charm, not 5...",5
20487,great location price view hotel great quick pl...,4
20488,"ok just looks nice modern outside, desk staff ...",2
20489,hotel theft ruined vacation hotel opened sept ...,1


In [4]:
lemmatizer = WordNetLemmatizer()
from nltk.tokenize import word_tokenize
nltk.download('punkt')
ps=PorterStemmer()
import nltk
nltk.download('wordnet')

[nltk_data] Error loading punkt: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading wordnet: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>


False

In [5]:
!pip install -U spacy


In [ ]:
conda update spacy

In [ ]:
pip install wasabi==0.8.2

In [ ]:
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
STOP_WORDS |= {"nt","hotel","room"}
print(STOP_WORDS)

In [ ]:
def text_clean(text):
    text=text.lower()
   
    text=re.sub("\[.*?\]","",text)
    text=re.sub("[%s]" % re.escape(string.punctuation),"",text)
    text=re.sub("\w*\d\w*","",text)
    text=re.sub("\n","",text)
    
    return text

In [ ]:
cleaned_1 = lambda x: text_clean(x)
data["Cleaned_Reviews"]=pd.DataFrame(data.Review.apply(cleaned_1))

In [ ]:
Reviews_1=data.copy()
Reviews_1.drop(["Review","Rating"],axis=1,inplace=True)
Reviews_1["Cleaned_Reviews"][6]
Reviews_1.Cleaned_Reviews

In [ ]:
corpus=[]

In [ ]:
import nltk
nltk.download('omw-1.4')

In [ ]:
for i in range  (0,len(Reviews_1)):
    review=re.sub("[^a-zA-Z]"," ",Reviews_1["Cleaned_Reviews"][i])
    
    review=review.split()
    review=[lemmatizer.lemmatize(word) for word in review if not word in STOP_WORDS]
    review=" ".join(review)
    corpus.append(review)

In [ ]:
corpus[6]

In [ ]:
data["Cleaned_Review_Lemmatized"]=corpus


In [ ]:
#Creating positive wordcloud

from wordcloud import WordCloud
  

In [ ]:

with open("positive-words.txt","r") as pos:
    positive = pos.read().split("\n")

In [ ]:
wordcloud1 = WordCloud(width = 3000, height = 2000, random_state=1, background_color='white', colormap='viridis', collocations=False, stopwords=positive).generate(str(corpus ))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud1)
plt.axis('off')


In [ ]:
#Creating negative wordcloud

with open("negative-words.txt","r") as neg:
    negative = neg.read().split("\n")


In [ ]:
wordcloud_neg = WordCloud(width = 3000, height = 2000, random_state=1, background_color='black', colormap='Pastel1', collocations=False,stopwords=negative).generate(str(corpus))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud_neg)


In [ ]:
pip install -U textblob

In [ ]:
#Polarity and subjectivity#

import textblob
from textblob import TextBlob

data["Polarity"]=data["Cleaned_Reviews"].apply(lambda x:TextBlob(x).sentiment.polarity)
data["Subjectivity"]=data["Cleaned_Reviews"].apply(lambda x:TextBlob(x).sentiment.subjectivity)


In [ ]:
#Printing 5 reviews with highest polarity

print("5 Random Reviews with Highest Polarity:")
for index,review in enumerate(data.iloc[data['Polarity'].sort_values(ascending=False)[:5].index]['Cleaned_Reviews']):
    print('Review {}:\n'.format(index+1),review)

In [ ]:
#Printing 5 reviews with negative polarity

print("5 Random Reviews with Lowest Polarity:")
for index,review in enumerate(data.iloc[data['Polarity'].sort_values(ascending=True)[:5].index]['Cleaned_Reviews']):
  print('Review {}:\n'.format(index+1),review)   

In [ ]:
#Frequency Distribution based on Polarity

plt.figure(figsize=(30,20),edgecolor="black")
plt.margins(0.05)
plt.xlabel("Polarity",fontsize=50)
plt.xticks(fontsize=30)
plt.ylabel("Frequency",fontsize=50)
plt.yticks(fontsize=30)
plt.hist(data["Polarity"],bins=50)
plt.title("Frequency Distribution based on Polarity",fontsize=60)
plt.show()

In [ ]:
#Pie plot of percentage of ratings

plt.figure(figsize=(30,10))
plt.title('Percentage of Ratings', fontsize=20)
data.Rating.value_counts().plot(kind='pie', labels=['Rating5', 'Rating4', 'Rating3', 'Rating2', 'Rating1'],
                              wedgeprops=dict(width=.7), autopct='%1.0f%%', startangle= -20, 
                              textprops={'fontsize': 15})


In [ ]:
#Sentiment distribution based on ratings

polarity_avg = data.groupby('Rating')['Polarity'].mean().plot(kind='bar', figsize=(30,20),color="green",edgecolor="orange")
plt.xlabel('Rating', fontsize=45)
plt.ylabel('Average Sentiment', fontsize=45)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title('Average Sentiment per Rating Distribution', fontsize=50)
plt.show()

In [ ]:
#Counting no of words in each review

data["Word Count"]=data["Cleaned_Review_Lemmatized"].apply(lambda x:len(str(x).split()))

In [ ]:
#Average no of words wrt ratings

word_avg=data.groupby("Rating")["Word Count"].mean().plot(kind="bar",figsize=(50,30),color="green",edgecolor="orange")
plt.xlabel('Rating',fontsize=35)
plt.ylabel("Average Count of Words",fontsize=35)
plt.xticks(fontsize=30)
plt.yticks(fontsize=30)
plt.title("Average Count of Words wrt Ratings",fontsize=50)
plt.show()

In [ ]:
#Counting no of letters in each review

data['review_len'] = data['Cleaned_Review_Lemmatized'].astype(str).apply(len)

In [ ]:
#Average no of letters wrt ratings

letter_avg = data.groupby('Rating')['review_len'].mean().plot(kind='bar', figsize=(50,30),color="green",edgecolor="orange")
plt.xlabel('Rating', fontsize=35)
plt.ylabel('Count of Letters in Rating', fontsize=35)
plt.xticks(fontsize=40)
plt.yticks(fontsize=40)
plt.title('Average Number of Letters wrt Rating ', fontsize=40)
plt.show()


In [ ]:
#Counting 100 most common words in our data

from nltk.probability import FreqDist
mostcommon = FreqDist(data["Cleaned_Review_Lemmatized"]).most_common(100)
wordcloud = WordCloud(width=1600, height=800, background_color='white').generate(str(corpus))
fig = plt.figure(figsize=(30,10), facecolor='white')
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis('off')
plt.title('Top 100 Most Common Words', fontsize=100)
plt.tight_layout(pad=0)
plt.show()


In [ ]:
polarity_positive_data=pd.DataFrame(data.groupby("Cleaned_Reviews")["Polarity"].mean().sort_values(ascending=True))


In [ ]:
plt.figure(figsize=(50,30))
plt.xlabel('Polarity',fontsize=35)
plt.ylabel('Reviews',fontsize=35)
plt.xticks(fontsize=35)
plt.yticks(fontsize=35)
plt.title('Polarity of Reviews',fontsize=50)
polarity_graph=plt.barh(np.arange(len(polarity_positive_data.index)),polarity_positive_data['Polarity'],color='purple',)



In [ ]:
#categorizing reviews in positive,negative and neutral#

def sentiment(x):
    if x<0:
        return 'negative'
    elif x==0:
        return 'neutral'
    else:
        return 'positive'

In [ ]:
data['polarity_score']=data['Polarity'].\
   map(lambda x: sentiment(x))

In [ ]:
plt.hist(data["polarity_score"],bins=10, color='orange', alpha=0.5, label='Value', edgecolor='black', linewidth=1)
plt.xlabel("Polarity Sentiments",fontsize=20)
plt.ylabel("Frequency",fontsize=20)
plt.xticks(fontsize=10)
plt.yticks(fontsize=10)
plt.title("Polarity Sentiments wrt Frequency",fontsize= 20)
plt.show()

In [ ]:
from collections import Counter


In [ ]:
#Most frequent words for rating 1

group_by = data.groupby('Rating')['Cleaned_Review_Lemmatized'].apply(lambda x: Counter(' '.join(x).split()).most_common(25))
group_by_0 = group_by.iloc[0]
words0 = list(zip(*group_by_0))[0]
freq0 = list(zip(*group_by_0))[1]
plt.figure(figsize=(50,30))
plt.bar(words0, freq0,color="green",edgecolor="orange")
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for Rating=1', fontsize=60)
plt.show()

In [ ]:
#Most frequent words for rating 2

group_by_1 = group_by.iloc[1]
words1 = list(zip(*group_by_1))[0]
freq1 = list(zip(*group_by_1))[1]
plt.figure(figsize=(50,30))
plt.bar(words1, freq1,color="green",edgecolor="orange")
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for Rating=2', fontsize=60)
plt.show()

In [ ]:
#Most frequent words for rating 3

group_by_2 = group_by.iloc[2]
words2 = list(zip(*group_by_2))[0]
freq2 = list(zip(*group_by_2))[1]
plt.figure(figsize=(50,30))
plt.bar(words2, freq2,color="green",edgecolor="orange")
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for Rating=3', fontsize=60)
plt.show()


In [ ]:
#Most frequent words for rating 4

group_by_3 = group_by.iloc[3]
words3 = list(zip(*group_by_3))[0]
freq3 = list(zip(*group_by_3))[1]
plt.figure(figsize=(50,30))
plt.bar(words3, freq3,color="green",edgecolor="orange")
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for Rating=4', fontsize=60)
plt.show()

In [ ]:
#Most frequent words for rating 5

group_by_4 = group_by.iloc[4]
words4 = list(zip(*group_by_4))[0]
freq4 = list(zip(*group_by_4))[1]
plt.figure(figsize=(50,30))
plt.bar(words4, freq4,color="green",edgecolor="orange")
plt.xlabel('Words', fontsize=50)
plt.ylabel('Frequency of Words', fontsize=50)
plt.yticks(fontsize=40)
plt.xticks(rotation=60, fontsize=40)
plt.title('Frequency of 25 Most Common Words for Rating=5', fontsize=60)
plt.show()

In [ ]:
#topic modelling using LDA#


In [ ]:
import re
import numpy as np
import pandas as pd
from pprint import pprint


In [ ]:
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel


In [ ]:
# spacy for lemmatization
import spacy


In [ ]:
pip install pyLDAvis

In [ ]:
conda install -c memex pyldavis

In [ ]:
#Term Frequency with Count Vetorizer

In [ ]:
# Stop word
stop_words_keywords = stopwords.words('english')


In [ ]:
# special additioanl stop words added for keyword extraction
stop_words_keywords.extend([
    "will", "always", "go", "one", "very", "good", "only", "mr", "lot", "two",
    "th", "etc", "don", "due", "didn", "since", "nt", "ms", "ok", "almost",
    "put", "pm", "hyatt", "grand", "till", "add", "let", "hotel", "able",
    "per", "st", "couldn", "yet", "par", "hi", "well", "would", "I", "the",
    "s", "also", "great", "get", "like", "take", "thank"
])


In [ ]:
# find the most frequent words in the data, extracting information about its content and topics.

In [ ]:
pip install Afinn

In [ ]:
import Afinn
from afinn import Afinn
from collections import Counter

In [ ]:
# Feature Extraction
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from imblearn.over_sampling import RandomOverSampler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
stops = set(stopwords.words('english') + ['com'])
co = CountVectorizer(stop_words=stop_words_keywords)
counts = co.fit_transform(corpus)
most_freq_terms = pd.DataFrame(counts.sum(axis=0),
                               columns=co.get_feature_names()).T.sort_values(
                                   0, ascending=False).head(25)

In [ ]:
most_freq_terms.plot(kind='bar', title='Unigram Frequency', figsize=(15, 10))


In [ ]:
# Now we can check for frequent bi-grams:
co = CountVectorizer(ngram_range=(2, 2), stop_words=stop_words_keywords)
counts1 = co.fit_transform(corpus)
bi_grams = pd.DataFrame(counts1.sum(axis=0),
                        columns=co.get_feature_names()).T.sort_values(
                            0, ascending=False).head(25)


In [ ]:
bi_grams.plot(kind='bar', title='bi-grams', figsize=(13, 10))

In [ ]:
# Now we can check for frequent tri-grams:
co = CountVectorizer(ngram_range=(3, 3), stop_words=stop_words_keywords)
counts1 = co.fit_transform(corpus)
tri_grams = pd.DataFrame(counts1.sum(axis=0),
                         columns=co.get_feature_names()).T.sort_values(
                             0, ascending=False).head(25)

In [ ]:
tri_grams.plot(kind='bar', title='tri-grams', figsize=(13, 10))


In [ ]:
#Term Frequency with TFIDF


In [ ]:
# Converting text into vectors by TFIDF
tfidf2 = TfidfVectorizer(norm="l2",analyzer='word', stop_words=stop_words_keywords,ngram_range=(1,1))
tfidf2_x = tfidf2.fit_transform(corpus)
most_freq_terms = pd.DataFrame(tfidf2_x.sum(axis=0),
                               columns=tfidf2.get_feature_names()).T.sort_values(
                                   0, ascending=False).head(25)

In [ ]:
most_freq_terms.plot(kind='bar',
                     title='most frequent terms & their frequency',
                     figsize=(15, 10))


In [ ]:
# Now we can check for frequent bi-grams:

tfidf2 = TfidfVectorizer(norm="l2",
                         analyzer='word',
                         stop_words=stop_words_keywords,
                         ngram_range=(2, 2))
tfidf2_x = tfidf2.fit_transform(corpus)
most_freq_terms = pd.DataFrame(
    tfidf2_x.sum(axis=0),
    columns=tfidf2.get_feature_names()).T.sort_values(0,
                                                      ascending=False).head(25)


In [ ]:
plt.rcParams.update({'font.size': 22})
most_freq_terms.plot(kind='bar', title='Bi-Gram', figsize=(15, 10))

In [ ]:
# Now we can check for frequent tri-grams:

tfidf2 = TfidfVectorizer(norm="l2",
                         analyzer='word',
                         stop_words=stop_words_keywords,
                         ngram_range=(3, 3))
tfidf2_x = tfidf2.fit_transform(corpus)
most_freq_terms = pd.DataFrame(
    tfidf2_x.sum(axis=0),
    columns=tfidf2.get_feature_names()).T.sort_values(0,
                                                      ascending=False).head(25)


In [ ]:
plt.rcParams.update({'font.size': 22})
most_freq_terms.plot(kind='bar', title='Tri-Gram', figsize=(15, 10))


In [ ]:
#Important Attribute Extraction from the Reviews
CountVectorizer


In [ ]:
cv2 = CountVectorizer(stop_words=stop_words_keywords,ngram_range=(1,2))
cv2_x = cv2.fit_transform(corpus)
tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
tfidf_transformer.fit(cv2_x)

In [ ]:
# get feature names
feature_names = cv2.get_feature_names()


In [ ]:
# fetch document for which keywords needs to be extracted
doc = corpus[532]


In [ ]:
# generate tf-idf for the given document
tf_idf_vector = tfidf_transformer.transform(cv2.transform([doc]))

In [ ]:
#Function for sorting tf_idf in descending order
from scipy.sparse import coo_matrix

In [ ]:
def sort_coo(coo_matrix):
    tuples = zip(coo_matrix.col, coo_matrix.data)
    return sorted(tuples, key=lambda x: (x[1], x[0]), reverse=True)


In [ ]:
def extract_topn_from_vector(feature_names, sorted_items, topn=10):
    """get the feature names and tf-idf score of top n items"""
    

In [ ]:
#Feature extraction by converting text in vector by using TFIDF

In [ ]:
tfidf = TfidfVectorizer(max_features=3000,
                       ngram_range=(1,3))


In [ ]:
# TFIDF for X-train
corpus_tfidf= tfidf.fit_transform(corpus)


In [ ]:
# To save & load models
from pickle import dump
from pickle import load



In [ ]:
# save the model to disk
filename = 'model_TFIDF.sav'

In [ ]:
dump(tfidf, open('model_TFIDF.sav', 'wb'))


####    X and Y Variable

In [ ]:
afinn=Afinn()

In [ ]:
# Affin Sentiment Score
score = [afn.score(item) for item in data['Cleaned_Reviews']]


In [ ]:
# Positive review = 1
# Negative review = 0

# Affin Sentiment Making it a two class classification problem
Affin_sentiment = [1 if score > 0 else 0 for score in score]

In [ ]:
# Affin Sentiment & Score added to the data frame
data['Affin_score'] = score
data['Affin_sentiment'] = Affin_sentiment

In [ ]:
# viewing only the negative reviews
data[data['Affin_sentiment'] == 0]['Cleaned_Reviews']

In [ ]:
# Creation of X and Y variable
X = corpus_tfidf
y = data['Affin_sentiment']

#Handling imbalanced data
We will have to handling the imbalanced data set by using SMOTE Technique

In [ ]:
# visualization of the imbalanced Y variable
sns.countplot(y)


In [ ]:
# the minority class will over sampled by 80%
stregegy = 0.8

# initializing of SMOTEN Sampling
over = SMOTE(sampling_strategy= stregegy,random_state=7)


In [ ]:
# Over Samplled X and Y variable 
X_over, y_over = over.fit_resample(X, y)

In [ ]:
sns.countplot(y_over)


Split the Data into Train and Test


In [ ]:
#Model Validation
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold



In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X_over, y_over,random_state=7,test_size=0.20,stratify=y_over)

In [ ]:
print("X_train = {}".format(X_train.shape))
print("X_test = {}".format(X_test.shape))
print("y_train = {}".format(y_train.shape))
print("y_test = {}".format(y_test.shape))


In [ ]:
# Model Building

In [ ]:
#Model Accuracy
from sklearn import metrics
from sklearn.metrics import accuracy_score, classification_report, roc_auc_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import plot_confusion_matrix, classification_report


In [ ]:
# custom Fuction to run the model with confusion matrix + Train and test accuracy + F1score
def model_metric(X_train, X_test, y_train, y_test, model, name):

    # predicted for X_train and X_test
    predict_train = model.predict(X_train)
    predict_test = model.predict(X_test)

    # Accuracy
    accuracy_train = model.score(X_train, y_train)
    accuracy_test = model.score(X_test, y_test)

    # F1score
    f1_score = metrics.f1_score(y_train, predict_train)

    print('Accuracy for Train set {}'.format(accuracy_train))
    print('Accuracy for Test set {}'.format(accuracy_test))
    print('Fi Score {}'.format(f1_score))
    
    print(classification_report(y_test, predict_test))

    plot_confusion_matrix(estimator=model,
                          X=X_test,
                          y_true=y_test,
                          cmap='Blues')
    plt.grid(False)
    plt.title('{} - Confusion Matrix on Test set'.format(name))

In [ ]:
 from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV


In [ ]:
#Logistic regression and fit the model
classifier = LogisticRegression()
classifier.fit(X_train,y_train)

In [ ]:
#Predict for X dataset
y_pred_train = classifier.predict(X_train)

In [ ]:
y_pred_train_df = pd.DataFrame({'actual': y_train,
                         'predicted_prob': y_pred_train})

In [ ]:
y_pred_train_df

In [ ]:
from sklearn.metrics import accuracy_score

print(f'Training-set accuracy score: {round(accuracy_score(y_train, y_pred_train),3)}')

In [ ]:
y_pred_test = classifier.predict(X_test)

In [ ]:
y_pred_test_df= pd.DataFrame({'actual': y_test,
                         'predicted_prob': y_pred_test})

In [ ]:
y_pred_test_df

In [ ]:
acc_val = accuracy_score(y_test, y_pred_test)
acc_val_rounded = round(acc_val,4)

In [ ]:
print(f'Testing-set accuracy score: {acc_val_rounded}')

In [ ]:
# To save & load models
from pickle import dump
from pickle import load
import pickle

In [ ]:
model = LogisticRegression()
model.fit(X_train, y_train)

In [ ]:
# save the model to disk
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))
 

In [ ]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, y_test)
print(result)